In [1]:
import numpy as np
import cvxpy as cp
from utils import parse_m_file, compute_PTDF


np.set_printoptions(precision=3, suppress=True, floatmode='fixed')


In [ ]:
from oct2py import Oct2Py

# Start Octave session
oc = Oct2Py()

# Load the MATPOWER case file
oc.eval("addpath('/Users/don_williams09/Downloads/Bi_Level_Opt/test_cases')")  # Adjust path to the folder containing the file
mpc = oc.case1354()

# Extract data
bus_data = mpc['bus']
gen_data = mpc['gen']
branch_data = mpc['branch']
gencost_data = mpc['gencost']


slack = int(bus_data[bus_data[:, 1] == 3, 0][0])
print(f"Slack bus: {slack}")

# Show dimensions or samples
print(f"Bus data shape : {bus_data.shape} ")
print(f"Generator data shape : {gen_data.shape}")
print(f"Branch data shape : {branch_data.shape} ")
print(f"Gencost data shape : {gencost_data.shape}")

num_buses = bus_data.shape[0]
num_generators = gen_data.shape[0]
num_lines = branch_data.shape[0]
num_branches = branch_data.shape[0]

# Data Extraction
Pd_base = bus_data[:, 2]
Pg_min = gen_data[:, 9]
Pg_max = gen_data[:, 8]
# Pg_max[0] = 400
# Pg_max[1] = 100
cost_coeff_true = gencost_data[:, 5]

branch_data_congested = branch_data.copy()
#branch_data_congested[:, 5] *= 0.7  # reduce limits by 30%

# PTDF = compute_PTDF(branch_data, bus_data)

def compute_PTDF(branch_data, bus_data, slack_bus=slack):
    """ Compute PTDF matrix given branch and bus data (MATPOWER format) """

    # Map bus IDs to matrix indices
    bus_ids = bus_data[:, 0].astype(int)
    id_to_index = {bus_id: idx for idx, bus_id in enumerate(bus_ids)}

    num_buses = len(bus_ids)
    num_lines = branch_data.shape[0]

    # Admittance matrix (B)
    B = np.zeros((num_buses, num_buses))
    for f_id, t_id, x in zip(branch_data[:, 0].astype(int),
                             branch_data[:, 1].astype(int),
                             branch_data[:, 3]):
        f = id_to_index[f_id]
        t = id_to_index[t_id]
        B[f, t] = -1 / x
        B[t, f] = -1 / x

    for i in range(num_buses):
        B[i, i] = -np.sum(B[i, :])

    # Remove slack bus row/col
    slack_index = id_to_index[slack_bus] if slack_bus in id_to_index else 0
    print(f"Slack index: {slack_index}")
    B_reduced = np.delete(np.delete(B, slack_index, axis=0), slack_index, axis=1)
    B_inv = np.linalg.pinv(B_reduced)

    # Compute B_line
    B_line = np.zeros((num_lines, num_buses))
    for idx, (f_id, t_id, x) in enumerate(zip(branch_data[:, 0].astype(int),
                                              branch_data[:, 1].astype(int),
                                              branch_data[:, 3])):
        f = id_to_index[f_id]
        t = id_to_index[t_id]
        B_line[idx, f] = 1 / x
        B_line[idx, t] = -1 / x

    # Remove slack column
    B_line_reduced = np.delete(B_line, slack_index, axis=1)

    # Compute PTDF
    PTDF = B_line_reduced @ B_inv

    return PTDF,id_to_index

PTDF, id_to_index = compute_PTDF(branch_data_congested, bus_data)


# Generator_to_bus incidence matrix
gen_to_bus = np.zeros((num_buses, num_generators))
for i, gen_bus_id in enumerate(gen_data[:, 0].astype(int)):
    bus_index = id_to_index[gen_bus_id]
    gen_to_bus[bus_index, i] = 1





Slack bus: 4231
Bus data shape : (1354, 13) 
 Generator data shape : (260, 10)
 Branch data shape : (1991, 13) 
 Gencost data shape : (260, 7)
Slack index: 639


In [3]:
print(f"maximum generation: {Pg_max}")
print(f"minimum generation: {Pg_min}")
print(f"cost coefficients: {cost_coeff_true}")
print(f"demand: {Pd_base}")
print(f"flow limits: {branch_data_congested[:, 5]}")

maximum generation: [1000.000  160.000  100.000  120.000 2000.000  280.000  540.000  600.000
  100.000  100.000   40.000  480.000  100.000   80.000  240.000  120.000
  100.000   33.580   80.000  100.000 2000.000  200.000 1000.000 2400.000
 2000.000  120.000  200.000  900.000  120.000 1200.000   80.000  100.000
 2000.000 1200.000  100.000  120.000  100.000  100.000   80.000   80.000
   80.000   80.000 2000.000 1000.000   40.000   41.060 2000.000  100.000
   31.920  100.000  912.780  472.160   40.000 1000.000 2000.000  200.000
  100.000  100.000  119.300  200.000   80.000  100.000  100.000  100.000
 2000.000   80.000   80.000  360.000  120.000  160.000 2000.000  120.000
  196.720 2000.000  100.000  160.000 2000.000  120.000   40.000    9.050
  214.860  120.000   40.000   80.000    6.830  493.350  200.000 1000.000
   40.000  240.000  100.000  120.000  540.000  100.000  100.000 1000.000
  100.000   80.000   80.000  135.270  120.000  120.000 1200.000  100.000
  900.000   80.000  600.000  10

In [ ]:
# Storage for scenario data
num_scenarios = 5
perturbation_scale = 0.25  # 10% perturbation
Pg_scenarios = []
lambda_slack_scenarios = []
nu_max_scenarios = []
nu_min_scenarios = []
mu_min_scenarios = []
mu_max_scenarios = []
Pd_scenarios = []

scenario_count = 0
np.random.seed(42)

while scenario_count < num_scenarios:
    Pd_perturbed = Pd_base * (1 + np.random.uniform(-perturbation_scale, perturbation_scale, size=num_buses))
    #Pd_perturbed = np.random.randint(0, 100, size=Pd_base.shape) + np.random.randint(0, 100, size=Pd_base.shape)/100
    Pd_perturbed[0] = 0  # slack bus
    Pd_scenarios.append(Pd_perturbed)
    
    Pg = cp.Variable(num_generators)
    P_inj = gen_to_bus @ Pg - Pd_perturbed
    
    # Exclude slack bus from PTDF
    indices = [i for i in range(num_buses) if i != id_to_index[slack]]
    P_inj_reduced = P_inj[indices]


    constraints = [
        cp.sum(Pg) == np.sum(Pd_perturbed),
        Pg >= Pg_min,
        Pg <= Pg_max,
        -branch_data_congested[:, 5] <= PTDF @ P_inj_reduced,
        PTDF @ P_inj_reduced <= branch_data_congested[:, 5]
    ]

    objective = cp.Minimize(cp.sum(cp.multiply(cost_coeff_true, Pg)))
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)

    if prob.status == 'optimal':
        scenario_count += 1
        Pg_scenarios.append(Pg.value)
        Pd_scenarios.append(Pd_perturbed)
        lambda_slack_scenarios.append(constraints[0].dual_value)
        nu_min_scenarios.append(constraints[3].dual_value)
        nu_max_scenarios.append(constraints[4].dual_value)
        mu_min_scenarios.append(constraints[1].dual_value)
        mu_max_scenarios.append(constraints[2].dual_value)
        print(f"Scenario {scenario_count}/{num_scenarios} generated.")
    else:
        print(f"Scenario not optimal. Status: {prob.status}")
        continue

Pg_scenarios = np.array(Pg_scenarios)
lambda_slack_scenarios = np.array(lambda_slack_scenarios)
nu_max_scenarios = np.array(nu_max_scenarios)
nu_min_scenarios = np.array(nu_min_scenarios)
mu_min_scenarios = np.array(mu_min_scenarios)
mu_max_scenarios = np.array(mu_max_scenarios)




Scenario not optimal. Status: infeasible
Scenario not optimal. Status: infeasible
Scenario not optimal. Status: infeasible


KeyboardInterrupt: 

In [12]:
print("lambda_slack_scenarios:\n", lambda_slack_scenarios)
print("Pg_scenarios:\n", Pg_scenarios)
print("Pd_scenarios:\n", Pd_scenarios)
print("nu_max_scenarios:\n", nu_max_scenarios)
print("nu_min_scenarios:\n", nu_min_scenarios)  

lambda_slack_scenarios:
 [-35.785 -37.111 -37.111 -36.038 -37.374]
Pg_scenarios:
 [[  -0.000   -0.000   -0.000   -0.000   -0.000  844.129    0.000   -0.000
    93.000    1.053 1936.793    0.000   -0.000   -0.000  297.000  316.669
    91.000   -0.000   -0.000    0.000  242.000   -0.000    0.000    0.000
     0.000  125.000  250.000 2465.000 1624.000   -0.000 1562.711  430.000
     0.000  460.000  554.000   -0.000  436.000    0.000  341.000  881.000
     0.000    0.000  855.961   -0.000  342.000    0.000  670.000 1401.000
   668.000  626.000  327.235   -0.000  543.000   -0.000    0.000  718.000
   150.000   -0.000 1233.000  557.031  203.000 1520.000    0.000    0.000
     0.000    0.000    0.000   84.000   -0.000]
 [  -0.000   -0.000   -0.000   -0.000   -0.000  846.201   -0.000   -0.000
    93.000    0.000 1658.555  321.000   -0.000   -0.000  297.000  356.320
    91.000   -0.000   -0.000  238.023  242.000   -0.000  240.993    0.000
     0.000  125.000  250.000 2465.000 1624.000   -0.000 

In [13]:
# Inverse Optimization (using multiple scenarios)
coef = cp.Variable(num_generators, nonneg=True)
loss = 0
BigM = 1e6

Pg_inv = cp.Variable((num_scenarios, num_generators))
lambda_slack_inv = cp.Variable(num_scenarios)   
nu_min_inv = cp.Variable((num_scenarios, num_lines))
nu_max_inv = cp.Variable((num_scenarios, num_lines))
mu_min_inv = cp.Variable((num_scenarios, num_generators))
mu_max_inv = cp.Variable((num_scenarios, num_generators))
epsilon_line = cp.Variable((num_scenarios, num_lines))
epsilon_gen = cp.Variable((num_scenarios, num_generators))

constraints_inv = []
stationarity = []
primal_feasibility = []
dual_feasibility = []
complementary = []

for t in range(num_scenarios):
    
    P_inj_inv = gen_to_bus @ Pg_inv[t] - Pd_scenarios[t]
    P_inj_reduced_inv = P_inj_inv[1:]
    flow_lines = PTDF @ P_inj_reduced_inv
    
    for gen in range(num_generators):
        # stationarity condition for generator
        stationarity.append(
            coef[gen] + lambda_slack_inv[t] + mu_max_inv[t][gen] - mu_min_inv[t][gen]      
                    + (nu_max_inv[t] - nu_min_inv[t]) @ PTDF @ gen_to_bus[1:, gen] == 0
        )
    
    primal_feasibility += [
        cp.sum(Pg_inv[t]) == np.sum(Pd_scenarios[t]),
        Pg_inv[t] >= Pg_min,
        Pg_inv[t] <= Pg_max,
        -branch_data[:, 5] <= flow_lines,
        flow_lines <= branch_data[:, 5]
    ]

    dual_feasibility += [
        mu_min_inv >= 0,
        mu_max_inv >= 0,
        nu_min_inv >= 0,
        nu_max_inv >= 0
    ]
    
    # Complementary slackness conditions
    for i in range(num_branches):
        # nu_min * (- branch_data_congested[:, 5] - PTDF @ P_inj_reduced) == 0
        if nu_min_scenarios[t][i] != 0:
            complementary.append(cp.abs(-branch_data_congested[i, 5] - flow_lines[i])<= epsilon_line[t][i])
            dual_feasibility.append(nu_max_inv[t][i] <= epsilon_line[t][i]) # check this
        # nu_max * (PTDF @ P_inj_reduced - branch_data_congested[:, 5]) == 0 
        elif nu_max_scenarios[t][i] != 0:
            complementary.append(cp.abs(flow_lines[i] - branch_data_congested[i, 5])<= epsilon_line[t][i])
            dual_feasibility.append(nu_min_inv[t][i] <= epsilon_line[t][i]) # check this
        else:     
            complementary.append(nu_min_inv[t][i] <= epsilon_line[t][i])
            complementary.append(nu_max_inv[t][i] <= epsilon_line[t][i])
           
    
    for i in range(num_generators):
        # mu_max * (Pg - Pg_max) == 0
        if Pg_scenarios[t][i] == Pg_max[i]:
            dual_feasibility.append(mu_max_inv[t][i] >= 0)
            dual_feasibility.append(mu_min_inv[t][i] <= epsilon_gen[t][i])
        # mu_min * (Pg_min - Pg) == 0
        elif Pg_scenarios[t][i] == Pg_min[i]:
            dual_feasibility.append(mu_min_inv[t][i] >= 0)
            dual_feasibility.append(mu_max_inv[t][i] <= epsilon_gen[t][i])
        else:
            dual_feasibility.append(mu_max_inv[t][i] <= epsilon_gen[t][i])
            dual_feasibility.append(mu_min_inv[t][i] <= epsilon_gen[t][i])
    
    # Loss function
    loss += cp.norm(Pg_inv[t] - Pg_scenarios[t], 2)**2
    loss += cp.norm(lambda_slack_inv[t] - lambda_slack_scenarios[t], 2)**2
    for i in range(num_generators):
        loss += cp.norm(mu_min_inv[t][i] - mu_min_scenarios[t][i], 2)**2
        loss += cp.norm(mu_max_inv[t][i] - mu_max_scenarios[t][i], 2)**2
        loss += epsilon_gen[t][i] * BigM
    for i in range(num_lines):      
        loss += cp.norm(nu_max_inv[t][i] - nu_max_scenarios[t][i], 2)**2
        loss += cp.norm(nu_min_inv[t][i] - nu_min_scenarios[t][i], 2)**2
        loss += epsilon_line[t][i] * BigM

constraints_inv += stationarity + primal_feasibility  + dual_feasibility + complementary

#constraints_inv.append(coef[2:5] == 0)

# loss += cp.sum_squares(Pg_inv - Pg_scenarios)
# loss += cp.sum_squares(lambda_slack_inv - lambda_slack_scenarios)
# loss += cp.sum_squares(mu_min_inv - mu_min_scenarios)
# loss += cp.sum_squares(mu_max_inv - mu_max_scenarios)
# loss += cp.sum_squares(nu_min_inv - nu_min_scenarios)
# loss += cp.sum_squares(nu_max_inv - nu_max_scenarios)


loss *= 1 / num_scenarios

inv_prob = cp.Problem(cp.Minimize(loss), constraints_inv)
inv_prob.solve(solver=cp.MOSEK, verbose=True)

if inv_prob.status == 'optimal':
    print("Inferred cost coefficients: ", coef.value)
    print("True cost coefficients: ", cost_coeff_true)
    print("Optimal cost: ", inv_prob.value)
else:
    print("Inverse problem not optimal: ", inv_prob.status)


                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) May 08 12:13:27 PM: Your problem has 7619 variables, 33950 constraints, and 0 parameters.
(CVXPY) May 08 12:13:28 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 08 12:13:28 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 08 12:13:28 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 08 12:13:28 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 08 12:13:29 PM: Compiling problem (target solver=MOSEK

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))

# Plot inferred and real cost coefficients
plt.step(range(len(coef.value)), coef.value, where='mid', label='Inferred Coefficients', linestyle='-')
plt.step(range(len(cost_coeff_true)), cost_coeff_true, where='mid', label='True Coefficients', linestyle='--')

# Add labels, legend, and title
plt.xlabel(f"Generator Index ({num_generators})", fontsize=8)
plt.ylabel('Cost Coefficient', fontsize=8)
plt.title(f"Inferred & True Cost Coefficients \n {num_buses} Buses - {num_scenarios} Scenarios", fontsize=12)
plt.legend(loc='upper right', fontsize=8)
plt.grid(True)

plt.savefig(f"/plots/case{num_buses}_scen{num_scenarios}.png", dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

print("Inferred cost coefficients: ", coef.value)
print("True cost coefficients: ", cost_coeff_true)

# Inverse Optimization Variables
print("loss:\n", loss.value)
print("Pg_inv:\n", Pg_inv.value)
print("lambda_slack_inv:\n", lambda_slack_inv.value)
print("nu_max_inv:\n", nu_max_inv.value)
print("nu_min_inv:\n", nu_min_inv.value)
print("mu_min_inv:\n", mu_min_inv.value)
print("mu_max_inv:\n", mu_max_inv.value)
print("epsilon_line:\n", epsilon_line.value)
print("epsilon_gen:\n", epsilon_gen.value)
